In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN


2024-09-29 20:42:51.690082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 20:42:52.005076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 20:42:52.093770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 20:42:52.584146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 20:42:59.436098: W tensorflow/compiler/tf2

In [2]:
df=pd.read_csv('tanmay_best_data_cleaned.csv')

In [4]:
df.drop(columns=['create_dt','time_of_day'],inplace=True)

In [ ]:
df.sortby('create_dt',inplace=True)

In [5]:
import numpy as np

def create_sequences(data, target_column, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:i + window_size].drop(columns=[target_column]).values)
        y.append(data.iloc[i + window_size][target_column])
    return np.array(X), np.array(y)

# Define the window size (e.g., 30 time steps)
window_size = 30

# Create sequences from the DataFrame
X, y = create_sequences(df, target_column='operation_kind_id', window_size=window_size)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the TCN model
def build_tcn_model(input_shape, num_classes):
    model = models.Sequential()

    # Temporal Convolutional Layers
    model.add(layers.Conv1D(filters=64, kernel_size=2, dilation_rate=1, padding='causal', activation='relu', input_shape=input_shape))
    model.add(layers.Conv1D(filters=64, kernel_size=2, dilation_rate=2, padding='causal', activation='relu'))
    model.add(layers.Conv1D(filters=64, kernel_size=2, dilation_rate=4, padding='causal', activation='relu'))

    # Global Average Pooling
    model.add(layers.GlobalAveragePooling1D())

    # Fully Connected Layer
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.2))

    # Output Layer for multi-class classification
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Model input shape: (window_size, number of features)
input_shape = (X.shape[1], X.shape[2])
num_classes = len(np.unique(y))

# Build the model
tcn_model = build_tcn_model(input_shape, num_classes)

# Train the model
tcn_model.fit(X, y, epochs=10, batch_size=64, validation_split=0.2)
